<a href="https://colab.research.google.com/github/dobrotvorn/DataCon23/blob/main/Mini_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Описание задания**
# Часть 1. Работа с данными

1) Просмотреть датасет на наличие выбросов и неверных данных

2) Разобраться в графиках из бейзлайна

3) Построить свои графики и выдвинуть по ним гипотезы

4) Отталкиваясь от этого, добавить некоторое количество новых дескрипторов при помощи rdkit (главное разобраться в коде, который это делает)

# Часть 2. Построение моделей

1) Построить свои модели на основе базовых регрессий из scikit-learn (минимум: обойти бейзлайн и попробовать хотя бы 3-4 модели
отличные от него)

2) Попробовать поработать с изменением параметров моделей и изменить соотношение тестовой/тренировочной выборки

3) Вывести графики feature importance полученных моделей

# Часть 3. Tensorflow

1) Провести эксперименты с архитектурами: количество слоев; функции активации; добавление дроп-аута/нормализации



## **Загрузка данных и библиотек**
# Импортируем и скачиваем библиотеки

In [ ]:
!pip install pymatgen
!pip install rdkit
!pip install gdown -- quiet
!pip install fancyimpute --quiet
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import plotly
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer
from IPython.display import display, HTML
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from fancyimpute import KNN
import pymatgen.core as mg
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps
from zipfile import ZipFile

In [ ]:
# тут захардкодим статичные переменные
global ordinal_encoder
global text_columns
global path2firstDb
global numerics
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
path2firstDb = '/content/firstDb.csv' if 'google.colab' in sys.modules else 'firstDb.csv'

# Скачиваем данные

In [ ]:
url1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR7whTOZIO16kGSfmtP3zHyXoRog0r6LVk_UZjpIhHwiRocAL5SIiqUyoBfpFXqe1fnyDsIOhTc7cjM/pub?gid=139006229&single=true&output=csv'
output1 = 'firstDb.csv'
gdown.download(url1, output1, quiet=True)

## **Часть 1. Работа с ДС**

**Описание параметров:**

SMILES - Представление молекулы в текстовом формате smiles.

Solubility - Растворимость вещества в воде.

MolLogP - десятичный логарифм коэффициента разделения.

MolMR -  Относительный молекулярный вес соединения.

HeavyAtomCount - Количество атомов не-водорода в молекуле.

NumRotatableBonds - Число поворотных связей в молекуле.

NumValenceElectrons - Количество  валентных электронов в соединении.

NumAromaticRings - Количество ароматических колец в молекуле.

TPSA - Топологическая площадь поверхности полярных атомов в соединении.

LabuteASA - площадь поверхности молекулы, доступная для взаимодействия с молекулами растворителя.

BertzCT - Индекс, предназначенный для количественной оценки “сложности” молекул.

MolWt - Молекулярный вес соединения.


In [ ]:
fdb = pd.read_csv(path2firstDb)
fdb.head(7)
fdb.info()

# Исключение выбросов

In [ ]:
cols = fdb.columns
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00']
sns.heatmap(fdb[cols].isnull(), cmap=sns.color_palette(colours))
# процентный список пропущенных данных
for col in fdb.columns:
    pct_missing = np.mean(fdb[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
# Проверка на дубликаты по SMILES
df_dedupped = fdb.drop_duplicates(subset='SMILES')
print(fdb.shape)
print(df_dedupped.shape)
# Дубликаты отсутствуют

In [ ]:
# Меняет значения выше четырёх медианных абсолютных отклонений (Фильтр Хэмпеля)
def hampel(vals_orig):
    vals = vals_orig.copy()
    difference = np.abs(vals.median()-vals)
    median_abs_deviation = difference.median()
    threshold = 4 * median_abs_deviation
    outlier_idx = difference > threshold
    vals[outlier_idx] = np.nan
    return(vals)

In [ ]:
# def quart(vals_orig):
#   vals = vals_orig.copy()
#   Q1 = fdb.quantile(q=.25)
#   Q3 = fdb.quantile(q=.75)
#   quartiles = vals[col].quantile([0.25, 0.75])
#   IQR = quartiles[0.75] - quartiles[0.25]
#   data_clean = vals[~((vals < (Q1-1.5*IQR)) | (vals > (Q3+1.5*IQR))).any(axis=1)]
#   return(data_clean)
# fdb = fdb.loc[:, 'Solubility':]
# fdb = quart(fdb)
# fdb.info()

In [ ]:
fdb = hampel(fdb)
fdb.info()

In [ ]:
sns.boxplot(data=fdb, x='Solubility')

In [ ]:
fdb = fdb.dropna(subset=['Solubility'])
fdb=fdb.reset_index(drop=True)

In [ ]:
fdb

In [ ]:
def get_molmr(smiles):
  new_mol=Chem.MolFromSmiles(smiles)
  a = Chem.Crippen.MolLogP(new_mol)
  return a

fdb['MolLogP'] = fdb['SMILES'].apply(get_molmr)
fdb.head()

In [ ]:
def get_molmr(smiles):
  new_mol=Chem.MolFromSmiles(smiles)
  a = Chem.Crippen.MolMR(new_mol)
  return a

fdb['MolMR'] = fdb['SMILES'].apply(get_molmr)
fdb.head()